<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/triplet/DL_Project_wandb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

--2021-07-05 18:14:26--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.152.137
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.152.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  79.08M  56.7MB/s    in 1.4s    

2021-07-05 18:14:28 (56.7 MB/s) - ‘dataset.zip’ saved [82925180/82925180]



In [2]:
!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 267 (delta 148), reused 146 (delta 67), pack-reused 0
Receiving objects: 100% (267/267), 10.79 MiB | 5.28 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [4]:
%cd /content/deeplearning_unitn
!git fetch origin 
!git checkout triplet

/content/deeplearning_unitn
Already on 'triplet'
Your branch is up to date with 'origin/triplet'.


In [ ]:
!nvidia-smi

Mon Jun 28 16:03:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
config = dict(
    wandb = True,
    device = "auto", # Select an specific device "auto" to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test", 
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    #train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    #test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    #queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    #attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    split = dict(
        full_training_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=128,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9,
    test_before_training=True,
    mAP_rank=15)

In [2]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from datasets.reid_dataset import Market1501
from cost_functions import OverallLossWrapper
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import FinetunedModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

/content/deeplearning_unitn
cuda:0


In [3]:
!pip install wandb -q
import wandb
if config["wandb"]:
  wandb.login()

wandb: Currently logged in as: deayalar (use `wandb login --relogin` to force relogin)


In [4]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    0) Split data
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    train_set, val_set, val_queries = split_data(config)
    
    model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
    id_ground_truth_dict = build_ground_truth(val_set, val_queries)

    print("Using "+ config["backbone"] + " as backbone")
    if config["test_before_training"]:
      test(model, val_loader, val_queries_loader, id_ground_truth_dict, config)

    train(model, train_loader, val_loader, criterion, optimizer, val_queries_loader, id_ground_truth_dict, config)

    test(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=True)

    return model

In [5]:
def build_ground_truth(val_set, val_queries):
    values = []
    for q in val_queries:
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))
    return ground_truth_dict


In [6]:
def split_data(config):
    """Returns a list with the names of theimages in each set"""
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries

def setup(train_set, val_set, val_queries, config):
    #Create pytorch Datasets
    train_composed = transforms.Compose([ transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225]),
                                          transforms.RandomErasing(p=0.6)])
    
    val_composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            images_list=train_set,
                            transform=train_composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         images_list=val_set,
                         transform=val_composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         images_list=val_queries,
                         transform=val_composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=2,
                                               drop_last=True)
                                               
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size=config["validation_batch_size"], 
                                             shuffle=False, 
                                             num_workers=2)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             batch_size=config["validation_batch_size"],
                                             shuffle=False, 
                                             num_workers=2)

    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")

    model = FinetunedModel(architecture=config["backbone"],
                           n_identities=len(train_dataset.unique_identities)).to(device)

    #This is a combination of the attributes classification loss and the triplet loss for identification
    criterion = OverallLossWrapper()
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, criterion, optimizer

In [7]:
def train(model, train_loader, val_loader, criterion, optimizer, val_queries_loader, id_ground_truth_dict, config):
    print("Training...")
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    if config["wandb"]:
         wandb.watch(model, criterion, log="all", log_freq=10)
    
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    for epoch in tqdm(range(config["epochs"])):
        model.train()
        for batch_idx, (inputs, identity, attributes) in enumerate(train_loader):
            loss = train_batch(inputs, identity, attributes, model, optimizer, criterion)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)

        test(model, val_loader, val_queries_loader, id_ground_truth_dict, config)


def train_batch(inputs, identity, attributes, model, optimizer, criterion):
    inputs = inputs.to(device)
    attributes = attributes.to(device)
    
    # Forward pass
    output_attrs, output_ids, output_features = model(inputs)

    # Apply the loss
    loss = criterion(output_attrs, attributes, output_features, identity, output_ids)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [8]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Epoch {epoch}: Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [9]:
def get_features_from_loader(model, loader):
    model.eval()
    with torch.no_grad():
        all_features = torch.zeros(len(loader.dataset), model.feature_size)
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                _, _, features = model(inputs)
                for in_batch, f in enumerate(features):
                    all_features[(batch_idx * loader.batch_size) + in_batch] = f
        return all_features


def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """
    top_k = get_topK_results(model, gallery_loader, queries_loader, config)
    #Build the dictionary to compute the mAP
    predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
    mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
    
    print(f"mAP: {mAP}")
    if config["wandb"]:
        wandb.log({"mAP": mAP})

def get_topK_results(model, gallery_loader, queries_loader, config):
  with torch.no_grad():
        
        # Compute the features for queries and gallery
        print("Computing gallery features...")
        gallery_features = get_features_from_loader(model, gallery_loader)
        print("Computing query features...")
        query_features = get_features_from_loader(model, queries_loader)
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        print("Computing cosine similarities...")
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0])
        for idx, q in enumerate(query_features):
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)
        
        print("Similarity matrix shape: " + str(sims_matrix.size()))
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])
        return top_k

In [10]:
def get_attributes_from_loader(model, loader):
    model.eval()

    all_predictions = np.empty(shape=[0, 27], dtype=np.byte)
    all_attrs = np.empty(shape=[0, 27], dtype=np.byte)

    with torch.no_grad():
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                outputs, _, _ = model(inputs)
                #print("attr:",attr)
                predictions = torch.empty(attr.size()[1], attr.size()[0])
                for attr_idx, output in enumerate(outputs):
                    if output.size()[1] == 1: #If the output is binary
                        pred = torch.round(torch.squeeze(output, 1))
                    else: #Otherwise it is multiclass
                        pred = torch.argmax(output, dim=1)
                    predictions[attr_idx] = pred

                predictions = torch.transpose(predictions, 0, 1).cpu().numpy()
                attr = attr.cpu().numpy()

                all_predictions = np.append(all_predictions, predictions, axis=0)
                #print("all_predictions shape: ", all_predictions.shape)
                all_attrs = np.append(all_attrs, attr, axis=0)
                #print("all_attrs shape: ", all_attrs.shape)
        return all_predictions, all_attrs

def test_attributes(model, loader, config):
    print("Computing attributes...")
    predictions, attr = get_attributes_from_loader(model, loader)
    print("pred shape: ", predictions.shape)
    print("attr shape: ", attr.shape)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    for i in range(0, predictions.shape[1]):
        y_true, y_pred = attr[:, i], predictions[:, i]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        if i == 0: #If it is age
            precision_list.append(precision_score(y_true, y_pred, average='macro'))
            recall_list.append(recall_score(y_true, y_pred, average='macro'))
            f1_score_list.append(f1_score(y_true, y_pred, average='macro'))
        else:
            precision_list.append(precision_score(y_true, y_pred))
            recall_list.append(recall_score(y_true, y_pred))
            f1_score_list.append(f1_score(y_true, y_pred))

    average_acc = np.mean(accuracy_list)
    average_precision = np.mean(precision_list)
    average_recall = np.mean(recall_list)
    average_f1score = np.mean(f1_score_list)

    print("accuracy_list: ", accuracy_list)
    print("precision_list: ", precision_list)
    print("recall_list: ", recall_list)
    print("f1_score_list: ", f1_score_list)

    print("average_acc: ", average_acc)
    print("average_precision: ", average_precision)
    print("average_recall: ", average_recall)
    print("average_f1score: ", average_f1score)

    if config["wandb"]:
            wandb.log({"accuracy_list": accuracy_list})
            wandb.log({"precision_list": precision_list})
            wandb.log({"recall_list": recall_list})
            wandb.log({"f1_score_list": f1_score_list})

            wandb.log({"average accuracy": average_acc})
            wandb.log({"average precision": average_precision})
            wandb.log({"average recall": average_recall})
            wandb.log({"average f1": average_f1score})

In [11]:
def test(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    print("Testing")
    model.eval()

    test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config)
        
    test_attributes(model, gallery_loader, config)

    if save_model :
      torch.save(model.state_dict(), "model")
      if config["wandb"]:
        wandb.save("model")

In [12]:
model = model_pipeline(config)

{'wandb': True, 'device': 'auto', 'train_root': '/content/dataset/train', 'test_root': '/content/dataset/test', 'queries_root': '/content/dataset/queries', 'attributes_file': '/content/dataset/annotations_train.csv', 'dataset': 'Market1501', 'backbone': 'resnet18', 'split': {'full_training_size': 0.8}, 'compose': {'resize_h': 224, 'resize_w': 224}, 'epochs': 1, 'training_batch_size': 128, 'validation_batch_size': 32, 'learning_rate': 0.01, 'weight_decay': 1e-06, 'momentum': 0.9, 'test_before_training': True, 'mAP_rank': 15}
Extract queries proportion: 0.11
Identities in train set: 600
Identities in validation set: 151
Train set size: 10214
Validation set size: 2469
Number of validation queries: 306
Number of attributes: 27
Backbone feature size: 512
Using resnet18 as backbone
Testing
Computing gallery features...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([306, 2469])
mAP: 0.10559833294934866
Computing attributes...



pred shape:  (2469, 27)
attr shape:  (2469, 27)
accuracy_list:  [0.2916160388821385, 0.6739570676387201, 0.6921830700688538, 0.4572701498582422, 0.3296881328473066, 0.6164439044147428, 0.46091535034426895, 0.5791818550020251, 0.9258809234507898, 0.46577561765897124, 0.09315512353179425, 0.5475901174564601, 0.11502632644795464, 0.8878088294856217, 0.7982989064398542, 0.8622924260834346, 0.3665451599837991, 0.1595787768327258, 0.6018631024706359, 0.6277845281490482, 0.0688537869582827, 0.995949777237748, 0.9935196435803969, 0.829485621709194, 0.330498177399757, 0.9210206561360875, 0.8286755771567437]
precision_list:  [0.24627935476516188, 0.2755905511811024, 0.21483375959079284, 0.096, 0.8672438672438673, 0.6178527354997944, 0.9209621993127147, 0.3967254408060453, 0.0, 0.4627858627858628, 0.09315512353179425, 0.3694390715667311, 0.10668855149774313, 0.00784313725490196, 0.08542713567839195, 0.0, 0.08751529987760098, 0.05555555555555555, 0.45, 0.03673938002296211, 0.03490328006728343, 0.

...


pred shape:  (2469, 27)
attr shape:  (2469, 27)
accuracy_list:  [0.7610368570271365, 0.8108545970028351, 0.6447954637505062, 0.8922640745240988, 0.8910490076954233, 0.8865937626569461, 0.9360064803564196, 0.7918185500202511, 0.9983799108950993, 0.8396111786148238, 0.9574726609963548, 0.8541919805589308, 0.9469420818144998, 0.9833940866747671, 0.9761036857027137, 0.8849736735520454, 0.9360064803564196, 0.9534224382341029, 0.8286755771567437, 0.955852571891454, 0.9716484406642365, 1.0, 0.9890643985419199, 0.8590522478736331, 0.8793033616848926, 0.964358039692183, 0.8622924260834346]
precision_list:  [0.19025921425678413, 0.7433628318584071, 0.2936893203883495, 0.02564102564102564, 0.8999577880962432, 0.9329608938547486, 0.9360064803564196, 0.7416331994645248, 0.0, 0.7985553772070626, 0.9370629370629371, 0.7512899896800825, 0.9928057553956835, 0.2857142857142857, 0.8343558282208589, 0.7045454545454546, 0.6510416666666666, 0.6016949152542372, 0.9202453987730062, 0.5211267605633803, 0.6808

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)



Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([306, 2469])
mAP: 0.41021015282700574
Computing attributes...



pred shape:  (2469, 27)
attr shape:  (2469, 27)
accuracy_list:  [0.7610368570271365, 0.8108545970028351, 0.6447954637505062, 0.8922640745240988, 0.8910490076954233, 0.8865937626569461, 0.9360064803564196, 0.7918185500202511, 0.9983799108950993, 0.8396111786148238, 0.9574726609963548, 0.8541919805589308, 0.9469420818144998, 0.9833940866747671, 0.9761036857027137, 0.8849736735520454, 0.9360064803564196, 0.9534224382341029, 0.8286755771567437, 0.955852571891454, 0.9716484406642365, 1.0, 0.9890643985419199, 0.8590522478736331, 0.8793033616848926, 0.964358039692183, 0.8622924260834346]
precision_list:  [0.19025921425678413, 0.7433628318584071, 0.2936893203883495, 0.02564102564102564, 0.8999577880962432, 0.9329608938547486, 0.9360064803564196, 0.7416331994645248, 0.0, 0.7985553772070626, 0.9370629370629371, 0.7512899896800825, 0.9928057553956835, 0.2857142857142857, 0.8343558282208589, 0.7045454545454546, 0.6510416666666666, 0.6016949152542372, 0.9202453987730062, 0.5211267605633803, 0.6808

## CREATE SUBMISSION FILES

In [24]:
reid_submission_file = "reid_test.txt"
test_images_list = sorted(os.listdir(config["test_root"]))
print(len(test_images_list))
queries_images_list = sorted(os.listdir(config["queries_root"]))
print(len(queries_images_list))

test_composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                       config["compose"]["resize_w"])),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

test_dataset = Market1501(root_dir=config["test_root"],
                          images_list = test_images_list,
                          test_dataset = True,
                          transform=test_composed)

query_dataset = Market1501(root_dir=config["queries_root"],
                             images_list = queries_images_list,
                             test_dataset = True,
                             transform=test_composed)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=config["validation_batch_size"], 
                                          shuffle=False, 
                                          num_workers=2)

query_loader = torch.utils.data.DataLoader(query_dataset, 
                                          batch_size=config["validation_batch_size"],
                                          shuffle=False, 
                                          num_workers=2)

19679
2248


In [16]:
#Load the model
model = FinetunedModel(architecture="resnet18", n_identities=600)
model.load_state_dict(torch.load("model"))
model.to(device)
model.eval()

top_k = get_topK_results(model, test_loader, query_loader, config)

Backbone feature size: 512
Computing gallery features...



Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([2248, 19679])


In [35]:
submission_id_dict = {queries_images_list[idx]: ', '.join([test_images_list[r] for r in results]) for idx, results in enumerate(top_k)}
f = open(reid_submission_file,"w")
f.write( str(submission_id_dict).replace("{", "").replace("}", "").replace("', ", "\n").replace("'", "") )
f.close()